In [1]:
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import string
from itertools import islice
from selenium.webdriver.common.keys import Keys
# from Feed import Feed
from icecream import ic
from typing import Iterator, Any, List

In [2]:
from Surfer import GetBrowser,RedditFeed,Feed
from Surfer.ScriptRunner import ScriptRunner

In [3]:
driver = GetBrowser()

In [4]:
def uuid():
    letters_digits = string.ascii_letters
    return ''.join(random.choice(letters_digits) for i in range(20))

In [5]:
def N(
  mean = 0.5
	, stddev = 0.01
	, bounds = None
):
	if bounds:
		return max(bounds[0], min(random.normalvariate(mean, stddev) , bounds[1]))
	else:
		return random.normalvariate(mean, stddev)

In [6]:
run_script = ScriptRunner(driver,'./Surfer/scripts/')

In [ ]:
class RedditPostFeed(Feed):
	def __init__(self, driver:webdriver.Chrome ,handle:str|None=None,verbose=False):
		super().__init__(name='Reddit Post Feed',driver=driver,handle=handle)
		# Store the WebDriver instance
		self.verbose = verbose
		self.focus()
		self.run_script('surfer.listen to clicks')()
		self.run_script('reddit.catch first comment',self.verbose)()

	def __iter__(self) -> Iterator[Any]:
		while True:
			self.focus()
			if not self.run_script('surfer.check cursor',verbose=self.verbose)():
				self.run_script('reddit.post.catch first comment',verbose=self.verbose)()
				continue
			self.run_script('surfer.scroll 2 cursor',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			for i in range(10):
				if self.run_script('surfer.check still scrolling')():
					time.sleep(0.1)
				else:
					break
				if i == 9:
					raise Exception('Not finished scrolling')
			yield self.run_script('reddit.post.parse comment',verbose=self.verbose)()
			self.run_script('reddit.post.cursor next')()

class RedditFeed(Feed):
	def __init__(self, driver:webdriver.Chrome,skip=True,handle:str|None=None,verbose=False):
		super().__init__(name='Reddit Feed',handle=handle,driver=driver)
		self.verbose = verbose
		if not skip:
			self.driver.get('https://www.reddit.com/')
			self.handle = self.driver.current_window_handle
			self.run_script('surfer.listen to clicks')()
		self.run_script('reddit.init_reddit',verbose=self.verbose)()
	

	def __iter__(self) -> Iterator[Any]:
		while True:
			self.focus()
			if not self.run_script('surfer.check cursor',verbose=self.verbose)():
				continue
			self.run_script('surfer.scroll 2 cursor',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			for i in range(10):
				if self.run_script('surfer.check still scrolling')():
					time.sleep(0.1)
				else:
					break
				if i == 9:
					raise Exception('Not finished scrolling')
			yield self.run_script('reddit.parse post',verbose=self.verbose)()
			self.run_script('surfer.cursor next')()

	def click_post(self, newTab=True):
		for i in range(10): # 10 retries
			self.focus()
			if i == 9:
				raise Exception('cursor not present')
			if not self.run_script('surfer.check cursor',verbose=self.verbose)():
				continue
			else:
				break
  
		id = uuid()
		self.run_script('surfer.mark cursor', verbose=self.verbose)(id)
		# Find the DOM element to click on
		sel = f'.{id}'
		element = driver.find_element(By.CSS_SELECTOR, sel)
		h3 = element.find_element(By.CSS_SELECTOR,'h3')
		element = element if h3 is None else h3
		actions = ActionChains(driver)
		actions \
			.click(element) \
			.perform()
			# .key_down(Keys.CONTROL) \
			# .key_up(Keys.CONTROL) \
		return RedditPostFeed(self.driver,self.handle,verbose=self.verbose)

	def close(self):
		self.focus()
		# Close the WebDriver instance
		self.driver.quit()


In [ ]:
rf = RedditFeed(driver,skip=False)
# rf = RedditFeed(driver,skip=True)

In [ ]:
# for x in islice(rf,15):
#   print(x)
#   time.sleep(2*max(1,len(x) * 0.01))

In [ ]:
e = rf.click_post()

In [ ]:
run_script('reddit.post.parse comment')()

In [9]:
class ChatGPT():
	def __init__(self, driver:webdriver.Chrome,skip=True,handle:str|None=None,verbose=False):
		self.name = 'GPT'
		self.handle = handle
		self.driver = driver
		self.verbose = verbose
		self.run_script = ScriptRunner(driver,'./Surfer/scripts/')
		if not skip:
			self.driver.get('https://chat.openai.com/')
			self.handle = self.driver.current_window_handle
			self.run_script('surfer.listen to clicks')()
	
	def ask(self,prompt:str):
		textarea = self.driver.find_element(By.CSS_SELECTOR,'textarea')

		JS_ADD_TEXT_TO_INPUT = """
			var elm = document.querySelector('textarea'), txt = arguments[0];
			elm.value += txt;
			elm.dispatchEvent(new Event('change'));
			"""
		prompts = prompt.split('\n')
		for line in prompts:
			self.driver.execute_script(JS_ADD_TEXT_TO_INPUT, line)
			textarea.send_keys(Keys.SHIFT + Keys.ENTER)
			
		self.driver.find_element(By.CSS_SELECTOR,'textarea + button').click()
		svg_path = 'M20.49 9A9 9 0 0 0 5.64 5.64L1 10m22 4l-4.64 4.36A9 9 0 0 1 3.51 15'
		WebDriverWait(self.driver, 30).until(
			EC.presence_of_element_located((By.CSS_SELECTOR, f'[d="{svg_path}"]'))
		)
		time.sleep(5)
		return run_script('chat.openai.get last answer')()



In [10]:
gpt = ChatGPT(driver,skip=True)

In [19]:
gpt.ask("Hi, sorry but I'm so bored right now!")

'No problem at all! Boredom can be persistent sometimes, but let\'s try to find something engaging for you. Here\'s another list of activities you can try:\n\nMeditate or Practice Mindfulness: Take a few moments to meditate or practice mindfulness. It can help reduce stress and increase focus.\n\nOnline Courses or Tutorials: Look for online courses or tutorials on topics you\'re interested in. There are countless resources available for learning new skills.\n\nVirtual Museum or Art Gallery Tours: Explore the world\'s museums and art galleries from the comfort of your home through virtual tours.\n\nStart a Journal or Blog: Write about your thoughts, experiences, or anything that comes to mind. It can be a great way to reflect and express yourself.\n\nVolunteer or Help Others: Look for virtual volunteering opportunities or simply help someone you know who may need assistance.\n\nPlan a Future Trip: Research and plan your dream vacation or a weekend getaway. It can be fun to envision futu